<img src="media/logo_psa.jpg" width="300">

<h1><center>Constructing SAMARA Data</center></h1>

## Imports

In [1]:
%load_ext autoreload
%autoreload 2
import os
import datetime
import numpy as np
import pandas as pd
from functools import reduce
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 100)

from pyspark.sql import functions as F

from distribution_cost.configuration import spark_config
from distribution_cost.configuration.app import AppConfig
from distribution_cost.configuration.data import DataConfig
from distribution_cost.infra import oracle
from distribution_cost.domain import kpis

/gpfs/user/e587247/dco00/conf/application.yml
/gpfs/user/e587247/dco00


## Connection to Exadata (Optional)

In [2]:
# Database uri
app_config = AppConfig()

db_uri = app_config.db_uri_jdbc
db_uri_cx_oracle = app_config.db_uri_cx_oracle

# Data Config
data_config = DataConfig()

data_config.vhls_perimeter

sites = data_config.vhls_perimeter["sites"]
start_date = data_config.vhls_perimeter["start_date"]
end_date = data_config.vhls_perimeter["end_date"]
genr_door = data_config.vhls_perimeter["genr_door"]

## Creating Spark Session

In [3]:
# Create spark session
spark_context, spark_session = spark_config.get_spark(app_name="app-reduce-brc10sinqtfv4",
                                                      executors=3, executor_cores=5, executor_mem='8g',
                                                      dynamic_allocation=True, max_executors=8)

# spark_session.conf.set("spark.sql.sources.partitionOverwriteMode","dynamic")

## Loading the tables from HDFS

In [4]:
df_sinqtfv4 = spark_session.read.load("/user/brc10/data/standardized/sinc0/sinqtfv4/")

## Parameters 

In [5]:
dateFrom = "01/08/2017"
dateTo = "01/02/2020"

In [6]:
dfsinqtfv4_reduced = df_sinqtfv4.where("DT_VD BETWEEN TO_DATE('{0}', 'dd/MM/yyyy') AND TO_DATE('{1}', 'dd/MM/yyyy')".format(dateFrom, dateTo))\
    .withColumn("year", F.year(F.col("DT_VD")))\
    .withColumn("month", F.month(F.col("DT_VD")))\
    .withColumn("day", F.dayofmonth(F.col("DT_VD")))

In [1]:
# dfsinqtfv4_reduced.explain(True)

### Writing the result in HDFS in partitions

In [8]:
dfsinqtfv4_reduced\
    .write\
    .mode("overwrite")\
    .partitionBy("year","month")\
    .parquet("hdfs:///user/e587247/data/landing/sinqtfv4_30month2/")

In [9]:
spark_session.stop()